### <font color='289C4E'>目录</font><a class='anchor' id='top'></a>
- [导入数据](#1)
- [标准化方向数据](#2)
- [合并数据](#3)
- [创建比赛摘要](#4)
- [探索性数据分析](#5)
- [建模](#6)
- [GIF/动画](#7)

# 1. 导入数据 <a class="anchor"  id="1"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# 读取 games.csv 文件
games_data = pd.read_csv('../../Dataset/games.csv')

# 读取 players.csv 文件
players_data = pd.read_csv('../../Dataset/players.csv')

# 读取 plays.csv 文件
plays_data = pd.read_csv('../../Dataset/plays.csv')

# 读取 tackles.csv 文件
tackles_data = pd.read_csv('../../Dataset/tackles.csv')

建立一个函数用于统计表格相关列的值信息

In [ ]:
def generate_summary_table(data):
    summary_table = pd.DataFrame(columns=['Column', 'Data Type', 'Missing Values', 'Missing %', 'Unique Values', 'Min', 'Max', 'Mean', 'Median'])
    
    for column in data.columns:
        data_type = str(data[column].dtype)
        
        missing_values = data[column].isnull().sum()
        missing_percentage = (missing_values / len(data)) * 100
        
        if data[column].dtype == 'object':
            min_value, max_value, mean_value, median_value = '', '', '', ''
        else:
            min_value = data[column].min()
            max_value = data[column].max()
            mean_value = data[column].mean()
            median_value = data[column].median()
        
        unique_values = data[column].nunique()  # 计算唯一值数量
        
        summary_table = summary_table.append({
            'Column': column,
            'Data Type': data_type,
            'Missing Values': missing_values,
            'Missing %': f'{missing_percentage:.2f}%',
            'Unique Values': unique_values,  # 添加唯一值数量
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value
        }, ignore_index=True)
    
    formatted_table = summary_table.style.set_properties(**{'text-align': 'center'})
    
    print("表格信息统计:")
    display(formatted_table)
    
    return summary_table


1. 比赛数据games.csv分析

In [ ]:
games_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

2. 球员数据players.csv分析

In [ ]:
players_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
players_data_summary = generate_summary_table(players_data)

3. 每场比赛数据plays.csv分析

In [ ]:
plays_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
plays_data_summary = generate_summary_table(plays_data)

4. 铲球数据tackles.csv分析

In [ ]:
tackles_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
tackles_data_summary = generate_summary_table(tackles_data)

# 2. 标准化数据 <a class="anchor"  id="1"></a>

1. games.csv

首先对该表格内容进行数据清洗与可视化

由之前信息可得此表中没有出现数据缺失的情况，因此可以直接处理

得分分布 - 直方图展示主队和客队得分的分布情况

In [ ]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

In [ ]:
# 创建直方图
fig = px.histogram(games_data, x=["homeFinalScore", "visitorFinalScore"], nbins=20, barmode="overlay",
                   labels={"value": "Score", "variable": "Team"}, title="Score Distribution")
fig.update_layout(xaxis_title="Score", yaxis_title="Frequency")
fig.show()


赛季中的队伍表现 - 折线图显示每支球队在赛季不同周次的得分情况

In [ ]:
# 根据赛季、周次和队伍分组并计算得分总和
team_scores = games_data.groupby(["season", "week", "homeTeamAbbr"])["homeFinalScore"].sum().reset_index()
team_scores = team_scores.rename(columns={"homeTeamAbbr": "Team", "homeFinalScore": "Score"})

# 获取所有队伍的列表
all_teams = team_scores["Team"].unique().tolist()

# 创建交互式折线图
fig = go.Figure()

for team in all_teams:
    team_data = team_scores[team_scores["Team"] == team]
    fig.add_trace(go.Scatter(x=team_data["week"], y=team_data["Score"], mode='lines', name=team))

# 设置布局
fig.update_layout(
    title="Team Performance in Different Weeks",
    xaxis_title="Week",
    yaxis_title="Score",
    legend=dict(orientation="h", y=-0.2),
    margin=dict(l=20, r=20, t=80, b=20)
)

fig.show()

从结果可以看出，并不是所有的队伍每个星期都参赛

赛季中比赛结果统计 - 饼图展示每个赛季主队和客队的胜利次数和比赛结果

In [ ]:
# 创建新列，表示主队和客队的胜利情况
games_data['home_win'] = games_data['homeFinalScore'] > games_data['visitorFinalScore']
games_data['visitor_win'] = games_data['homeFinalScore'] < games_data['visitorFinalScore']

# 统计每个赛季中主队和客队的胜利次数
season_results = games_data.groupby(['season']).agg({
    'home_win': 'sum',
    'visitor_win': 'sum'
}).reset_index()

# 重塑数据以符合饼图格式
season_results_melted = season_results.melt(id_vars='season', var_name='Result', value_name='Wins')

# 显示饼图
fig = px.pie(season_results_melted, values='Wins', names='Result', title='Season Results: Home Team vs Visitor Team Wins',
             hover_data=['Wins'], labels={'Result': 'Game Result'})
fig.show()

看来主场确实有优势啊

接下来进行数据清理的工作

In [ ]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

分析结果发现这里面存在着大量的非数值文本，这会使数据很难处理，因此我们要适当的舍弃或者对这些文本进行重新编码。接下来我们看看这些列分别代表了什么数据

这些字段的含义如下：

- **gameId（比赛ID）**：比赛的唯一标识符（数字）
- **season（赛季）**：比赛所属的赛季（数字）
- **week（周次）**：比赛所在的周次（数字）
- **gameDate（比赛日期）**：比赛日期（时间，月/日/年）
- **gameTimeEastern（比赛开始时间）**：比赛开始时间（时间，时:分:秒，东部标准时间）
- **homeTeamAbbr（主队三字母代码）**：主队的三字母代码（文本）
- **visitorTeamAbbr（客队三字母代码）**：客队的三字母代码（文本）
- **homeFinalScore（主队最终得分）**：主队在比赛中的总得分（数字）
- **visitorFinalScore（客队最终得分）**：客队在比赛中的总得分（数字）

这些字段描述了每场比赛的详细信息，包括比赛的标识、赛季、比赛日期和时间、参赛队伍及其得分情况。

* 其中赛季（season）肯定是一个无关值，因为这些数据都属于一个赛季，可以抛弃
* gameTimeEastern可能不太重要，因为是比赛举行的时间，稍后可以用热力图分析一下
* 而对主队客队，我们可以采用编码的方式来对这些队伍进行编码来剔除文本信息。

由于热力图只能处理数值信息，因此我们应先将队名，比赛日期和比赛时间进行编码

In [ ]:
# 获取所有队伍的列表，并按字母顺序排序
teams = sorted(set(games_data['homeTeamAbbr'].unique()) | set(games_data['visitorTeamAbbr'].unique()))

# 创建队伍编码映射
team_encoding = {team: code for code, team in enumerate(teams, 1)}

# 对 'homeTeamAbbr' 和 'visitorTeamAbbr' 进行编码
games_data['homeTeamAbbr'] = games_data['homeTeamAbbr'].map(team_encoding)
games_data['visitorTeamAbbr'] = games_data['visitorTeamAbbr'].map(team_encoding)

# 对 'gameDate' 进行日期编码
games_data['gameDate'] = pd.to_datetime(games_data['gameDate'])
games_data['gameDate'] = (games_data['gameDate'] - games_data['gameDate'].min()).dt.days + 1

# 对 'gameTimeEastern' 进行小时编码
games_data['gameTimeEastern'] = pd.to_datetime(games_data['gameTimeEastern']).dt.hour + \
                                pd.to_datetime(games_data['gameTimeEastern']).dt.minute / 60 + \
                                pd.to_datetime(games_data['gameTimeEastern']).dt.second / 3600

# 输出队名与编码的对应关系
for team, code in team_encoding.items():
    print(f"Team: {team} -> Code: {code}")

# 输出编码后的结果
print(games_data[['season', 'week', 'homeTeamAbbr', 'visitorTeamAbbr', 'homeFinalScore', 'visitorFinalScore', 'gameDate', 'gameTimeEastern']])

对于队名我采用了一个编码规则，按照字母排列编码
比赛日期也采用了相似的编码，比赛时间编码则是直接转化为小时数，比如20:00:00就编码为20

首先drop掉season，再画出热力图，看看比赛时间会不会对比赛的胜负结果产生影响。

In [ ]:
# 剔除 'season', 'home_win', 'visitor_win' 列
games_df_without_season = games_data.drop(columns=['season', 'home_win', 'visitor_win'])

# 计算相关性矩阵
correlation_matrix = games_df_without_season.corr()

# 创建热力图
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

fig.update_layout(
    title='Correlation Heatmap of Columns (Season Column Excluded)',
    xaxis_title='Columns',
    yaxis_title='Columns',
    annotations=annotations
)

fig.show()

发现gameId,week和gameDate强相关，这是因为比赛的时间安排决定了这几项必然强相关，相反，其他数据项的相关性并不大，我们可以放心剔除week和gameDate来减少数据维度，保留gameId，因为可能要用这项数据连接其他表

In [ ]:
games_data_cleaned = games_data.drop(columns=['season', 'home_win', 'visitor_win', 'week', 'gameDate'])
games_data_cleaned.head()  # 数据清理完成

至此，该表的数据清理完成

2. players.csv

首先对该表格内容进行公制化转换，转换成我们可以处理的数据

In [ ]:
players_data.head()

身高：米  体重：千克  出生日期：转化为年龄

In [ ]:
# 转换身高为厘米
players_data['height'] = players_data['height'].apply(lambda x: int(x.split('-')[0]) * 30.48 + int(x.split('-')[1]) * 2.54)

# 转换体重为公斤
players_data['weight'] = players_data['weight'] * 0.453592

# 计算年龄
players_data['birthDate'] = 2022 - pd.to_datetime(players_data['birthDate']).dt.year

# 输出转换后的结果
print(players_data)

In [ ]:
# 调用函数并打印生成的表格
players_data_summary = generate_summary_table(players_data)

由之前的统计信息可知，此表在生日值上是有缺失的，因此很多人的年龄不知道，并且缺失值达到了28.46%，不能采用中值或者均值进行替换，因此我们不得不舍弃该数据列，尽管在体育上年龄对于球员表现是重要因素。

而且球员姓名也有重名，因此我们不能对球员姓名进行编码，只能使用nflId来标识球员

In [ ]:
# 删除 'birthDate' 列
players_data.drop(['birthDate'], axis=1, inplace=True)

# 输出结果
print(players_data)

接下来我们进行可视化分析

In [ ]:
# 创建直方图统计身高每一厘米的频率
height_counts = players_data['height'].value_counts().sort_index()

fig = go.Figure(data=[go.Bar(x=height_counts.index, y=height_counts.values)])
fig.update_layout(
    title='Height Distribution',
    xaxis_title='Height (cm)',
    yaxis_title='Frequency'
)
fig.show()

In [ ]:
# 创建直方图统计体重
fig_weight = px.histogram(players_data, x='weight', title='Weight Distribution')
fig_weight.update_xaxes(title='Weight')
fig_weight.show()

In [ ]:
# 创建直方图不同位置的球员数量
fig_position = px.histogram(players_data, x='position', title='Player Position Distribution')
fig_position.update_xaxes(title='Position')
fig_position.show()

In [ ]:
# 创建箱线图得到位置与体重的关系
fig_box = px.box(players_data, x='position', y='weight', title='Position vs Weight')
fig_box.update_xaxes(title='Position')
fig_box.update_yaxes(title='Weight')
fig_box.show()

In [ ]:
# 计算各个球员位置的平均身高和体重
avg_height_weight = players_data.groupby('position').agg({'height': 'mean', 'weight': 'mean'}).reset_index()

# 绘制柱状图展示不同位置球员的平均身高和体重
fig_avg = px.bar(avg_height_weight, x='position', y=['height', 'weight'], 
                 barmode='group', title='Average Height and Weight by Position')
fig_avg.update_xaxes(title='Position')
fig_avg.update_yaxes(title='Value')
fig_avg.show()

In [ ]:
# 计算BMI指数
players_data['bmi'] = players_data['weight'] / ((players_data['height'] / 100) ** 2)

# 绘制BMI指数的直方图
fig_bmi = px.histogram(players_data, x='bmi', title='BMI Distribution')
fig_bmi.update_xaxes(title='BMI')
fig_bmi.show()

可视化分析完成后，接下来对其collegeName和position两列进行编码，将姓名drop掉使用nflId标识球员

In [ ]:
# 获取所有学院和职位的列表，并按字母顺序排序
college_names = sorted(players_data['collegeName'].unique())
positions = sorted(players_data['position'].unique())

# 创建学院和职位编码映射
college_encoding = {college: code for code, college in enumerate(college_names, 1)}
position_encoding = {position: code for code, position in enumerate(positions, 1)}

# 对 'collegeName' 和 'position' 进行编码
players_data['collegeName'] = players_data['collegeName'].map(college_encoding)
players_data['position'] = players_data['position'].map(position_encoding)

# 删除 'displayName' 列
players_data.drop(['displayName'], axis=1, inplace=True)

# 输出编码后的结果
print(players_data)